In [1]:
import socket
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.hashes import SHA256

# Function to encrypt data using the RSA public key
def encrypt_data(data, public_key):
    encrypted_data = public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_data

# Function to hash data using SHA-256
def hash_data(data):
    digest = hashes.Hash(SHA256())
    digest.update(data)
    return digest.finalize()

# Create an RSA key pair for the client
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Serialize the public key
public_key_pem = private_key.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)

# Create a socket and connect to the server
host = 'localhost'
port = 12339
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((host, port))

# Send the client's public key to the server
client_socket.send(public_key_pem)

# Receive the server's public key
server_public_key_pem = client_socket.recv(2048)

# Load the server's public key
server_public_key = serialization.load_pem_public_key(server_public_key_pem)

# Wait for the key exchange to complete
print("Key exchange completed. Sending encrypted and hashed message...")

# Message to be hashed, concatenated, and encrypted
message = b'This is a secret message to be encrypted.'
hash_value = hash_data(message)

# Concatenate the message and the hash
message_to_send = message + b'\n' + hash_value

# Encrypt the message using the server's public key
encrypted_message = encrypt_data(message_to_send, server_public_key)
print(encrypted_message)
# Send the encrypted message to the server
client_socket.send(encrypted_message)

# Close the socket
client_socket.close()


Key exchange completed. Sending encrypted and hashed message...
b'\r\xb5\xd5\x9e7\xa2\xde\x84\xa7riB\xb7)\xe8|\x04)\xad_\xaaYf-.\nZ\xad\x04U?\xe8lD\xf1\xee\xc2b\xa0\xc0\x18\x18\x03\xb9S\xce\xee\xfd\x16v\xc549\xd20:\x8d\xaf\xfb\x19\xb3b\xb7d\xd7+t\x85w\x8d_\x1c\x8d\x80\x8e\x0fSP\xdb\xdc7OS:\xfb\xa9\x01\x05g\x96\x9f\xad\xf3K\xf8_\xffP\xf4\x034;1\xee\xbd\xc0\xe0\xe0\xb7U\x12\xd4\xe9h\xa0\x1b6>\x95,\xd5I\xa0\xf4\xb7\xd0\xec\x8e\xa5\x00\xeb\xda\x86P\xc5\xd9\xa5`\xbd\xe0\xa0t\xeb\xc5\xfe:l\xe92\x9313\xa3o\x97\x81\xf4/v\xe7\x12O\x0c\x98\xf2J.\x88\xdd\xa6:\xd4\x0b\xb3.\xcb\x82\xd1X\xfc;Q\x8e\xed.\xaf\xa5\xce#\xad4l`s\x0c\xb3?\xa8\x01\x15\xa5\xcf\x13\x7f{\x95\xd3\xf8\xab\x8b.fV\xf9Lviu\x8b\xa4E\x0f\xab\xe2mP\xda\x91\x88gR\x98Y.\xc0\x7fi\xf6\xe5\xbb\xd63\xb65\xfb\x91\x1c@\x83\xd01G\xe6\xa8\x91\xae'
